# 연구 목표

- 데이터 
  - MNIST 손글씨 이미지 데이터 사용
- 목적
  - 손글씨 이미지 분류(0-9)
- 엔진
  - 텐서플로우
- 모델링
  - 딥러닝, CNN 모델
- 평가
  - 예측모델 생성, 정확도를 통해 평가 (분류의 평가지표)

# 데이터 확보

- 출처
  - 텐서플로우의 케라스에서 제공되는 MNIST 데이터를 사용

In [1]:
import tensorflow as tf
%tensorflow_version 1.x

In [2]:
mnist = tf.keras.datasets.mnist.load_data(path='mnist.npz')

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
type(mnist), len(mnist), len(mnist[0]), len(mnist[1])

(tuple, 2, 2, 2)

In [4]:
len(mnist[0][0]), len(mnist[0][1]), len(mnist[1][0]), len(mnist[1][1])

(60000, 60000, 10000, 10000)

In [5]:
# numpy.array임을 확인했으니 shape찍어본다........
mnist[0][0].shape, mnist[0][1].shape, mnist[1][0].shape, mnist[1][1].shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [6]:
mnist[0][1][:2], mnist[0][0][:2]

(array([5, 0], dtype=uint8), array([[[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]],
 
        [[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8))

In [0]:
# 케라스에서 제공하는 데이터는 쉽게 로드가 되지만, 원본 데이터여서
# 가공할 부분이 존재 => 가공된 데이터를 받아서 처리 (1.x)
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
mnist = input_data.read_data_sets('./data/mnist/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [9]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [10]:
# 훈련 데이터
mnist.train.images.shape, mnist.train.labels.shape

((55000, 784), (55000, 10))

In [11]:
# 테스트 데이터
mnist.test.images.shape, mnist.test.labels.shape

((10000, 784), (10000, 10))

# 데이터 준비

In [0]:
import numpy as np

In [13]:
# 이미지 1개당 feature의 크기
pixels = mnist.train.images.shape[1]
pixels

784

In [14]:
# 레이블  feature의 크기
nums = mnist.train.labels.shape[1]
nums

10

In [15]:
# 이미지 1개당 가로 혹은 세로 크기 (정사각형이므로 동일)
pixel_wh = int(np.sqrt(pixels))
pixel_wh

28

In [16]:
# 정답 레이블에서 원래값 0~9까지 찾아내는 방법
# array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]]) => 7 
# 나중에 정답을 출력을 위해

# np.argmax() => 최대값의 인덱스 반환!!
# np.where ?? => 특이값 인덱스 출력?
mnist.train.labels[0].argmax(), np.where(mnist.train.labels[0])[0][0]

(7, 7)

# 데이터 분석 (생략)

# 모델 구축, 딥러닝

## 1. 모델 설계, 레이어 설계 -> 모델링

In [17]:
'''
layers
  L 입력층
  L 중간층
    L 합성곱층
    L 풀링층
    L 합성곱층
    L 풀링층
    L 전결합층
    L 드롭아웃층
  L 출력층
'''

'\nlayers\n  L 입력층\n  L 중간층\n    L 합성곱층\n    L 풀링층\n    L 합성곱층\n    L 풀링층\n    L 전결합층\n    L 드롭아웃층\n  L 출력층\n'

## 2. 데이터 플로우 그래프 구축

### 입력층

- x
- 손글씨 이미지가 데이터로 주입된다
- 플레이스홀더로 구성
- shape : (None, 784=pixels)

In [18]:
x = tf.placeholder( tf.float32, shape=(None, pixels), name='x' )
x

<tf.Tensor 'x:0' shape=(?, 784) dtype=float32>

### 합성곱층 1

- 외부에서 이전단계 레이어에서 데이터가 흘러들어온다
- 데이터 주입? ( X )
- 구성원
  - W : 공용 파라미터, 가중치 => 필터, 커널
  - b : 편향
  - S : stride
  - p : padding

In [0]:
# 가중치 필터 W 생성 함수
# 가중치 필터 W는 언제나 3x3의 크기를 가지는 것이 아니다
# shape : W의 shape
# name : W의 이름 (텐서보드에서 해당 텐서들을 확인/구분용으로 사용)
def makeWeight(shape, name):
  # 변수 초기값 -> 절단평균분포를 통해서 난수값으로 초기화
  # 0.1은 설정값 -> 가중치 필터의 값은 일단 난수로 설정
  init_W = tf.truncated_normal(shape, stddev=0.1)
  # W를 생성
  W = tf.Variable( init_W, name='W_'+name)
  # 가중치 필터(커널) W를 리턴
  return W

In [0]:
def makeWeightVariable(shape, name):
  init_W = tf.truncated_normal(shape, stddev=0.1)
  W = tf.Variable( init_W, name='W_'+name)
  return W

In [0]:
# 편향 생성함수
def makeBiasVariable( shape, name ):
  # 상수로 값을 하나 고정하여서 적용
  init_b = tf.constant( 0.1, shape=[shape] )
  b      = tf.Variable( init_b, name='b_' + name )
  return b

In [22]:
# 합성곱층 생성함수
def makeConv2d(x, W, name):
  # conv2d = tf.nn.conv2d(x, filter=W, strides=[1,1,1,1], padding="VALID", name= 'conv_'+name)
  conv2d = tf.nn.conv2d(x, filter=W, strides=[1,1,1,1], padding="SAME", name= 'conv_'+name)
  return conv2d

''' 형태를 잘 맞추자!!!
tf.nn.conv2d(
  input  : [
            batch,      1회 훈련시 들어오는 데이터의 총 갯수, 이미지 총 갯수
            in_height,  이미지의 세로 크기
            in_width,   이미지의 가로 크기
            in_channels 입력이미지의 채널수, 1(흑백) or 3(RGB)
           ]
  filters: [
            filter_height, 필터의 세로  cur) 3
            filter_width,  필터의 가로  cur) 3
            in_channels,   입력 채널 수 cur) 1 
            out_channels   출력 채널 수, 알아서 설정(마음대로 설정, 설정값)
           ]
  strides: An int or list of ints that has length 1, 2 or 4
           정수값만, 정수값의 리스트로 구성
           구성값은 1 or 2 or 4
           [
            batch  batch와 depth는 일반적으로 1을 사용
            w      w와 h는 일반적으로 같은 값부여
            h      w와 h는 일반적으로 같은 값부여
            depth  batch와 depth는 일반적으로 1을 사용
           ] => [1,1,1,1] / [1,2,2,1] / [1,3,3,1] ...
  padding: Either the string "SAME" or "VALID"
           특성맵을 보정한다. 자르다 보면 외곽선 부분이 부족하여 필터작업이 안될수 있음.
           이를 보정하는 설정
           통상적으로는 0으로 세팅
            -> "SAME"  : 같은 크기로 보정
               "VALID" : 유효한 부분만 보정
)
'''

' 형태를 잘 맞추자!!!\ntf.nn.conv2d(\n  input  : [\n            batch,      1회 훈련시 들어오는 데이터의 총 갯수, 이미지 총 갯수\n            in_height,  이미지의 세로 크기\n            in_width,   이미지의 가로 크기\n            in_channels 입력이미지의 채널수, 1(흑백) or 3(RGB)\n           ]\n  filters: [\n            filter_height, 필터의 세로  cur) 3\n            filter_width,  필터의 가로  cur) 3\n            in_channels,   입력 채널 수 cur) 1 \n            out_channels   출력 채널 수, 알아서 설정(마음대로 설정, 설정값)\n           ]\n  strides: An int or list of ints that has length 1, 2 or 4\n           정수값만, 정수값의 리스트로 구성\n           구성값은 1 or 2 or 4\n           [\n            batch  batch와 depth는 일반적으로 1을 사용\n            w      w와 h는 일반적으로 같은 값부여\n            h      w와 h는 일반적으로 같은 값부여\n            depth  batch와 depth는 일반적으로 1을 사용\n           ] => [1,1,1,1] / [1,2,2,1] / [1,3,3,1] ...\n  padding: Either the string "SAME" or "VALID"\n           특성맵을 보정한다. 자르다 보면 외곽선 부분이 부족하여 필터작업이 안될수 있음.\n           이를 보정하는 설정\n           통상적으로는 0으로 세팅\n            -> "SAME"  : 같은 크

#### 합성곱층 연결

- x, W, b등을 연결하여 활성화 맵으로 출력하겠끔 데이터 플로우 그래프를 정의한다
- 텐서보드상에서 이 함성곱을 하나의 노드로 표현하기 위해, name_scope 하나의 이름이 미치는 공간, 관계, 형태를 규정

In [0]:
# 합성곱층 1 생성, 입력대비 출력까지의 모든 관계(그래프)를 표현
with tf.name_scope('conv1') as scope:
  # W
  # [필터의높이, 필터의가로, 입력채널수, 출력채널수(설정한다)]
  # 출력채널수를 32라고 지정하면, 이미지 한장을 넣어서, 결과물이 32개가 나온다
  # 1을 제외하고 5, 5, 32는 모두 설정값이다(실험치이다)
  W = makeWeightVariable( [5, 5, 1, 32 ], 'conv1' )
  # b
  b = makeBiasVariable( 32, 'conv1' )
  # x : 입력층 (None, 784) => (batch, h, w, channel)
  # 입력층의 모양을 tf.nn.conv2d에서 원하는 모양으로 맞춰준다
  x_imgs  = tf.reshape( x, (-1, pixel_wh, pixel_wh, 1 ) )
  # conv1
  h_conv1 = tf.nn.relu( makeConv2d( x_imgs, W, 'conv1' ) + b )
  #h_conv1 = tf.nn.relu( 피처맵 )
  # h_conv1 : activation Map이다

In [24]:
h_conv1

<tf.Tensor 'conv1/Relu:0' shape=(?, 28, 28, 32) dtype=float32>

In [25]:
h_conv1.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28), Dimension(32)])

<hr>

- 이미지의 수량
  - 합성곱층을 통과하면 1개의 이미지가 32개가 된다.
  - 이유는 가중치 필터에서 출력채널 수를 32로 했기 때문에 수량이 증가했다.
- 이미지의 크기
  - 현재 결과물은 (?, 28, 28, 32)
  - 이미지의 세로, 가로, 크기가 원본과 동일  
    => padding="SAME"
  - padding="VALID" => (?, 24, 24, 32)

<hr>

### 풀링층 1 

- 특성(특성맵, 활성화맵)을 강화한다.
- 최대/평균/최소 풀링 제공 및 구현 가능
- 샘플링 표현으로도 묘사
- stride, padding 없다  
  (명시적인 커널용 tride, padding은 없다.)
  - 풀링의 수행은 shape가 작아지는 결과를 반환한다.
  - 내부적으로 풀링 연산을 수행하기 위해서는 얼만큼 이동하여서 값을 추출해야 하는지 보폭의 양이 존재해야 한다.
  - stride가 있다면, padding도 존재해야 한다.
- 입력 : h_conv1 (?,28,28,32)

In [32]:
# 풀링 함수 => 최대 풀링
def makeMaxPooling( x ):
  # 결과물의 포맷 : NHWC ( 현재 상황의 기본값 )
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME' )

'''
  input   : 4-D tensor, 이전 단계ㅡ이 산출물, [ batch, h, w, channels ]
  ksize   : [batch, h, w, 출력 채널의 수] => 기본 커널에 대비해서 W는 없다
            이중에 최대값, 평균값, 최소값 등을 선택
            => 데이터 1개 대상 => 각 차원의 크기 지정
            => 최대풀링값이 1개 나온다 => 4번째 1의 의미
            => 입력이 1개           => 첫번째 1의 의미
            => 가운데 2,2만 신경쓰면 된다.
  strides : [batch, w, h, depth] / batch=depth, w or h = 1,2,4
            => 가운데 w, h가 최종 산출물의 크기에 영향을 미친다
  padding : 'SAME', 'VALID'
'''


"\n  input   : 4-D tensor, 이전 단계ㅡ이 산출물, [ batch, h, w, channels ]\n  ksize   : [batch, h, w, 출력 채널의 수] => 기본 커널에 대비해서 W는 없다\n            이중에 최대값, 평균값, 최소값 등을 선택\n            => 데이터 1개 대상 => 각 차원의 크기 지정\n            => 최대풀링값이 1개 나온다 => 4번째 1의 의미\n            => 입력이 1개           => 첫번째 1의 의미\n            => 가운데 2,2만 신경쓰면 된다.\n  strides : [batch, w, h, depth] / batch=depth, w or h = 1,2,4\n            => 가운데 w, h가 최종 산출물의 크기에 영향을 미친다\n  padding : 'SAME', 'VALID'\n"

In [0]:
# 풀링층 1 생성
with tf.name_scope('pool1') as scope:
  h_pool1 = makeMaxPooling( h_conv1 )

In [46]:
h_pool1 # shape=(?, 14, 14, 32)

<tf.Tensor 'pool1_5/MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

### 합성곱층 2

- 입력 : h_pool1
- 출력 : h_conv2 (?,14,14,64)

In [47]:
int(h_pool1.shape[1])

14

In [0]:
name_conv2 = 'conv2'
with tf.name_scope(name_conv2) as scope:
  # [h, w, 입력채널수=이전단계 채널수, 출력채널수=최종결과채널수]
  W_conv2 = makeWeightVariable( [5, 5, 32, 64 ], name_conv2 )
  b_conv2 = makeBiasVariable( 64, name_conv2 )
  # x reshape과정은 필요없다!!
  h_conv2 = tf.nn.relu( makeConv2d( h_pool1, W_conv2, name_conv2 ) + b_conv2 )

In [56]:
h_conv2

<tf.Tensor 'conv2_8/Relu:0' shape=(?, 14, 14, 64) dtype=float32>

### 풀링층 2 

- 입력 : h_conv2
- 출력 : h_pool2 => (?,7,7,64)

In [0]:
with tf.name_scope('pool2') as scope:
  h_pool2 = makeMaxPooling( h_conv2 )

In [58]:
h_pool2 # shape=(?, 7, 7, 64) 

<tf.Tensor 'pool2/MaxPool:0' shape=(?, 7, 7, 64) dtype=float32>

### 전결합층

### 드롭아웃층

### 출력층

### 학습 플로우 작성

### 예측 플로우 작성

### 평가 관련 플로우 작성(정확도)

## 3. 실행(학습, 예측, 평가)

# 시스템 통합(생략)